In [1]:
! pip install datasets transformers evaluate -q
! pip install tensorflow
! pip install torch torchvision
! pip install matplotlib
! pip install tf-keras
! pip install scikit-learn
! pip install tqdm
# ! pip install autoawq

In [1]:
from datasets import load_from_disk
from transformers import pipeline, AutoTokenizer
import tqdm

# Read the BOLD JSON files into a dataset
load_path='./prompts/sample_bold_prompts'
bold_prompts = load_from_disk(load_path)

# Load text generation pipeline and model
model = "mistralai/Mistral-7B-v0.1"
text_generation = pipeline("text-generation", model=model)
tokenizer = AutoTokenizer.from_pretrained(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
def generate_text(prompts, max_length=100, do_sample=False, pad_token_id=502256):
	model_continuations=[]
	# Iterate through prompts and generate text
	for prompt in tqdm.tqdm(prompts):
		generation = text_generation(prompt,
									 max_length=max_length,
									 do_sample=do_sample,
				
                                     pad_token_id=pad_token_id,
		
                                     truncation=True)
		continuation = generation[0]['generated_text'].replace(prompt,'').lstrip()
		model_continuations.append(continuation)
	return model_continuations

In [ ]:
# Select only first instance of prompt, some prompts have multiple sentences
toxic_prompts = [p['prompts'] for p in bold_prompts]

# Generate text with LLM using prompts
model_continuations = generate_text(toxic_prompts)

print('Generated '+ str(len(model_continuations))+ ' continuations')


# Update dataset with generations
bold_prompts = bold_prompts.add_column(f'generated_prompts_{model}', model_continuations)

 99%|██████████████████████████████████████████████████████████████████████████████▏| 99/100 [4:12:49<01:25, 85.98s/it]

In [ ]:
# Save resulting prompts to disk
out_name = f'./generated_data/prompts_{model}'
bold_prompts.save_to_disk(out_name)